<a href="https://colab.research.google.com/github/Srilatha23/NNDL/blob/main/NNDL_Assignment5_5A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
data=pd.read_csv('/content/drive/My Drive/Colab Notebooks/LD.csv')
data.head()

,Date,Time,Load (kW),Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,01.09.2018,0:00,5551.82208,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1:00,4983.17184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2:00,4888.39680,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3:00,5072.95872,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,4:00,5196.25980,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
X = []
Y = []
for i in range(24, data.shape[0]):
    X.append(data['Load (kW)'].iloc[i-25]) # one hour and one day before
    Y.append(data['Load (kW)'].iloc[i])

In [9]:
data = pd.DataFrame({'Prev Day Load' : X, 'Pres Day Load' : Y})
data.head()

,Prev Day Load,Pres Day Load
0,6938.37468,4931.26380
1,5551.82208,4775.53968
2,4983.17184,4713.81120
3,4888.39680,4689.02628
4,5072.95872,4844.75040


In [10]:
Xmax = np.max(data['Prev Day Load'])
Xmin = np.min(data['Prev Day Load'])
Ymax = np.max(data['Pres Day Load'])
Ymin = np.min(data['Pres Day Load'])
data['Prev Day Load'] = (data['Prev Day Load'] - Xmin) / (Xmax - Xmin)
data['Pres Day Load'] = (data['Pres Day Load'] - Ymin) / (Ymax - Ymin)
data.head()

,Prev Day Load,Pres Day Load
0,0.651650,0.284300
1,0.397877,0.255799
2,0.293800,0.244501
3,0.276454,0.239965
4,0.310234,0.268466


In [11]:
x = data['Prev Day Load']
y = data['Pres Day Load']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(x, y, test_size=0.1, random_state=10)

In [12]:
m = np.random.uniform(-2, 2)
c = np.random.uniform(-2, 2)
eta = 0.1
epochs = 500
gm = 0.9
Em2 = 0
Ec2 = 0
e = 1E-6

# Loop for epochs
for _ in range(epochs):
    
    for i in range(Xtrain.shape[0]):
        # Calculating Grad M
        gradM = -1 * (Ytrain.iloc[i] - m * Xtrain.iloc[i] - c) * Xtrain.iloc[i]
    
        # Calculating Grad C
        gradC = -1 * (Ytrain.iloc[i] - m * Xtrain.iloc[i] - c)
        
        # Calculating updated values of Em2 and Ec2
        Em2 = (gm*Em2) + ((1-gm)*(gradM ** 2))
        Ec2 = (gm*Ec2) + ((1-gm)*(gradC ** 2))
        
        # Updating m and c values
        m -= (eta * gradM) / ((e + Em2) ** 0.5)
        c -= (eta * gradC) / ((e + Ec2) ** 0.5)

# Print model parameters after training
print(f'm = {m}\nc = {c}')

m = 0.5301773463582954
c = 0.06091882796981383


In [13]:
train_pred = [m * Xtrain.iloc[i] + c for i in range(Xtrain.shape[0])]
test_pred = [m * Xtest.iloc[i] + c for i in range(Xtest.shape[0])]
train_pred = [i * (Ymax - Ymin) + Ymin for i in train_pred]
test_pred = [i * (Ymax - Ymin) + Ymin for i in test_pred]
trainY = [i * (Ymax - Ymin) + Ymin for i in Ytrain]
testY = [i * (Ymax - Ymin) + Ymin for i in Ytest]

In [14]:
# Training Error Calculation
training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print(f'MAE : {training_MAE}')
print(f'MSE : {training_MSE}')
print(f'RMSE : {training_RMSE}')

# Testing Error Calculation
testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {testing_MAE}')
print(f'MSE : {testing_MSE}')
print(f'RMSE : {testing_RMSE}')

Training Error :
MAE : 997.2928224924361
MSE : 1511874.7042977347
RMSE : 1229.5831424908747
Testing Error :
MAE : 963.7638493264467
MSE : 1381168.1014774719
RMSE : 1175.231084288308


In [15]:
pd.DataFrame({'Actual Values' : trainY, 'Predicted Values' : train_pred})

,Actual Values,Predicted Values
0,6694.42248,4974.640215
1,6241.43520,3928.449253
2,5028.06528,4884.376390
3,6094.28448,5428.934529
4,7306.09560,5351.992924
...,...,...
1939,3960.91080,4028.944411
1940,5538.72816,4856.541875
1941,5812.76520,6002.070980
1942,4949.65764,4469.354526


In [17]:
x = float(input('Enter the load at present hour on previouus day : '))
x = (x - Xmin) / (Xmax - Xmin)
prediction = m * x + c
prediction = (prediction * (Ymax - Ymin)) + Ymin
print('Predicted load at present hour :', prediction)

Enter the load at present hour on previouus day : 6354
Predicted load at present hour : 5288.6152480307455
